In [2]:
import datamuse
import requests
import numpy as np
import random
import string
import pandas as pd

In [5]:
api = datamuse.Datamuse()
headers = {
    'x-rapidapi-host': "wordsapiv1.p.rapidapi.com",
    'x-rapidapi-key': "f9e88b40a3mshe89fe995823038bp1f4ab2jsn4f97b67d0ad0"
    }

In [6]:
with open('words.txt', 'r') as f:
    lines = f.read().split(',')
list_lines = []
for item in lines:
    list_lines.append(item.translate({ord(c): None for c in string.whitespace}))
print(len(list_lines))

997


In [57]:
list_random_words = list_lines

In [58]:
print(len(list_random_words))
print(len(list_lines))

397
397


In [60]:
# Get 3 words that have the same rhyme and 1 word that has consonant match using Datamuse API
# http://www.datamuse.com/api/
for word in list_random_words:
    list_wrong = api.words(rel_cns=word, max=1)
    if len(list_wrong) < 1:
        continue
    list_right = api.words(rel_rhy=word, max=3)
    if len(list_right) < 3:
        continue
    answer = list_wrong[0]['word']
    list_words = [answer]
    for item in list_right:
        list_words.append(item['word'])
    random.shuffle(list_words)
    dict_questions = {
        'choice0' : list_words[0],
        'choice1' : list_words[1],
        'choice2' : list_words[2],
        'choice3' : list_words[3],
        'answer'  : list_words.index(answer),
    }
    # Get word's frequency to compute the difficulty using Words API
    # https://www.wordsapi.com/
    list_diff = []
    for item in list_words:
        url_frequency = f"https://wordsapiv1.p.rapidapi.com/words/{item}/frequency"
        response = requests.request("GET", url_frequency, headers=headers)
        data = response.json()
        try:
            f = data['frequency']['zipf']
            list_diff.append(7 - f)
        except:
            list_diff.append(7)
    diff_avg = sum(list_diff) / (len(list_diff) - 1)
    dict_questions.update({'diff' : diff_avg})
    list_questions.append(dict_questions)

In [64]:
len(list_questions)

893

In [62]:
df = pd.DataFrame(list_questions)

In [63]:
df.to_csv('./resource/rhymes_questionbank.csv')
df.head()

,choice0,choice1,choice2,choice3,answer,diff
0,abate,route,out,doubt,0,3.350000
1,sauce,loss,cross,accresce,3,4.836667
2,contract,impact,abstract,irked,3,5.280000
3,octave,attractive,proactive,interactive,0,5.083333
4,productivity,creativity,activator,proclivity,2,5.846667


In [5]:
df = pd.read_csv('./resource/rhymes_questionbank.csv')

In [8]:
df.head(10)

,Unnamed: 0,choice0,choice1,choice2,choice3,answer,diff
0,0,abate,route,out,doubt,0,3.350000
1,1,sauce,loss,cross,accresce,3,4.836667
2,2,contract,impact,abstract,irked,3,5.280000
3,3,octave,attractive,proactive,interactive,0,5.083333
4,4,productivity,creativity,activator,proclivity,2,5.846667
5,5,sad,bad,aid,pad,2,3.130000
6,6,off-road,blade,trade,aid,0,5.033333
7,7,laughter,thereafter,efta,rafter,2,6.280000
8,8,train,plane,vein,agone,3,4.903333
9,9,sage,engage,gauge,edge,3,4.360000
